In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from statsmodels.tsa.api import VAR
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
dataset = pd.read_csv("Ts-OHLC.csv")
dataset

,Open,High,Low,Close
0,1410.60,1427.90,1408.30,1415.10
1,1421.00,1626.60,1416.15,1607.40
2,1632.55,1673.90,1613.05,1626.20
3,1627.75,1627.75,1574.60,1579.05
4,1580.00,1639.50,1565.50,1595.65
...,...,...,...,...
2220,202.90,207.80,201.35,205.00
2221,206.00,206.85,202.05,202.95
2222,203.90,203.90,199.35,201.00
2223,200.00,222.00,196.00,218.35


In [2]:
column = "Close"

In [3]:
from sklearn.preprocessing import MinMaxScaler
Ms=MinMaxScaler()
data1 = Ms.fit_transform(dataset[[column]])
print("Len:" , data1.shape)

Len: (2225, 1)


In [4]:
training_size = round(len(data1) * 0.95)
print(training_size)

X_train = data1[:training_size]
X_test = data1[training_size:]
print("X_train length:", X_train.shape)
print("X_test length:", X_test.shape)

y_train = data1[:training_size]
y_test = data1[training_size:]
print("y_train length:", y_train.shape)
print("y_test length:", y_test.shape)

2114
X_train length: (2114, 1)
X_test length: (111, 1)
y_train length: (2114, 1)
y_test length: (111, 1)


In [5]:
performance={"Model":[],"RMSE":[],"MaPe":[],"Lag":[],"Test":[]}

In [6]:
listt = ["Close", "High", "Open", "Low"]
datasetTwo = dataset[listt].values 
datasetTwo

array([[1415.1 , 1427.9 , 1410.6 , 1408.3 ],
       [1607.4 , 1626.6 , 1421.  , 1416.15],
       [1626.2 , 1673.9 , 1632.55, 1613.05],
       ...,
       [ 201.  ,  203.9 ,  203.9 ,  199.35],
       [ 218.35,  222.  ,  200.  ,  196.  ],
       [ 212.35,  220.45,  219.65,  211.55]])

In [7]:
test_obs = 28
train = datasetTwo[:-test_obs]
test = datasetTwo[-test_obs:]

In [8]:
train

array([[1415.1 , 1427.9 , 1410.6 , 1408.3 ],
       [1607.4 , 1626.6 , 1421.  , 1416.15],
       [1626.2 , 1673.9 , 1632.55, 1613.05],
       ...,
       [ 225.6 ,  227.8 ,  224.9 ,  222.45],
       [ 224.8 ,  229.  ,  227.4 ,  223.25],
       [ 228.85,  231.4 ,  227.  ,  226.95]])

In [9]:
test

array([[225.4 , 231.95, 230.4 , 222.2 ],
       [213.85, 225.  , 225.  , 211.7 ],
       [217.3 , 219.4 , 211.  , 206.85],
       [216.6 , 223.  , 219.35, 215.8 ],
       [216.7 , 218.4 , 217.7 , 215.15],
       [208.05, 215.25, 215.  , 206.95],
       [199.4 , 206.75, 205.35, 192.15],
       [202.85, 207.  , 199.95, 199.95],
       [204.5 , 208.9 , 204.65, 203.05],
       [205.35, 206.  , 205.7 , 203.4 ],
       [204.35, 207.8 , 205.  , 203.5 ],
       [202.5 , 206.9 , 206.9 , 202.  ],
       [212.4 , 214.4 , 207.  , 204.1 ],
       [213.5 , 215.8 , 213.  , 210.5 ],
       [212.8 , 215.  , 213.5 , 211.25],
       [216.45, 219.25, 213.  , 212.1 ],
       [214.65, 218.25, 218.25, 213.1 ],
       [212.95, 214.7 , 214.7 , 211.65],
       [211.1 , 213.9 , 213.9 , 210.  ],
       [207.8 , 213.  , 213.  , 207.  ],
       [204.25, 208.  , 208.  , 203.05],
       [194.55, 200.  , 200.  , 190.3 ],
       [199.65, 204.05, 196.  , 196.  ],
       [205.  , 207.8 , 202.9 , 201.35],
       [202.95, 

In [10]:
for i in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]:
        model = VAR(train)
        results = model.fit(i)
        print('Order =', i)
        print('AIC: ', results.aic)
        print('BIC: ', results.bic)
        print()

Order = 1
AIC:  17.46805661920327
BIC:  17.519918120137966

Order = 2
AIC:  17.144732363091535
BIC:  17.238118123327677

Order = 3
AIC:  17.147708041702938
BIC:  17.282649265642515

Order = 4
AIC:  17.14882729921318
BIC:  17.32535522939668

Order = 5
AIC:  17.11977250561702
BIC:  17.337918422787507

Order = 6
AIC:  17.037619573610925
BIC:  17.297414796778224

Order = 7
AIC:  16.981340530479187
BIC:  17.28281641698422

Order = 8
AIC:  16.985477286455943
BIC:  17.32866523203521

Order = 9
AIC:  16.96932138080675
BIC:  17.35425281965693

Order = 10
AIC:  16.948085953037886
BIC:  17.374792357880587



In [11]:
x = model.select_order(maxlags=12)
order = x.selected_orders["aic"]
result = model.fit(order)

In [12]:
lagged_Values = train[-order:]
pred = result.forecast(y=lagged_Values, steps=28) 
preds = pd.DataFrame(pred, columns=listt)
preds.to_csv("varforecasted_{}.csv".format(test_obs))

In [13]:
rmse = round(mean_squared_error(test, pred, squared=False))
mape = mean_absolute_percentage_error(test, pred)

performance["Model"].append(listt)
performance["RMSE"].append(rmse)
performance["MaPe"].append(mape)
performance["Lag"].append(order)
performance["Test"].append(test_obs)
    
perf = pd.DataFrame(performance)

In [14]:
perf

,Model,RMSE,MaPe,Lag,Test
0,"[Close, High, Open, Low]",15,0.066959,11,28
